In [1]:
import warnings
import numpy as np
import pandas as pd
import copy
from pathlib import Path
import os

import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import pytorch_forecasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

C:\Users\sahab\anaconda3\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (SMAPE). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


In [2]:
def zdorovaya_dura(
    data: pd.DataFrame,
    batch_size: int,
    target: str,
    index_col: str,
    max_prediction_length: int,
    time_varying_unknown_reals: list,
    group_by: list,
    epochs: int):
    
    training = TimeSeriesDataSet(
        data=data, 
        time_idx=index_col,
        target=target,
        group_ids=group_by,
        add_relative_time_idx=True,
        time_varying_unknown_reals=time_varying_unknown_reals,
        max_prediction_length=max_prediction_length,
        min_encoder_length=1,
        max_encoder_length=max_prediction_length,
#         allow_missing_timesteps=True
    )
    
    validation = TimeSeriesDataSet.from_dataset(training, data
                                                , predict=True, stop_randomization=True)
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10
                                              , num_workers=0)
    
    lr_logger = LearningRateMonitor()
    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4
                                        , patience=10, verbose=False, mode="min")
    logger = TensorBoardLogger("lightning_logs")

    trainer = pl.Trainer(
        max_epochs=epochs,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.1,
        limit_train_batches=30,
        callbacks=[lr_logger, early_stop_callback],
        logger=logger,
    )

    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.01,
        hidden_size=16,
        attention_head_size=1,
        dropout=0.1,
        hidden_continuous_size=8,
        loss=QuantileLoss(),
        log_interval=25,
        output_size=7,
        reduce_on_plateau_patience=4,
    )
    
    trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    )
    
    return tft, tft.predict(data)

In [3]:
data_for_forecasting_0_prepared = pd.read_csv(r"C:\Users\sahab\Desktop\slovarik\сочная_фтмишница_это_все_для_тебя\сочная_фтмишница_это_все_для_тебя_0.csv")
data_for_forecasting_1_prepared = pd.read_csv(r"C:\Users\sahab\Desktop\slovarik\сочная_фтмишница_это_все_для_тебя\сочная_фтмишница_это_все_для_тебя_1.csv")
data_for_forecasting_2_prepared = pd.read_csv(r"C:\Users\sahab\Desktop\slovarik\сочная_фтмишница_это_все_для_тебя\сочная_фтмишница_это_все_для_тебя_2.csv")
data_for_forecasting_3_prepared = pd.read_csv(r"C:\Users\sahab\Desktop\slovarik\сочная_фтмишница_это_все_для_тебя\сочная_фтмишница_это_все_для_тебя_3.csv")
data_for_forecasting_4_prepared = pd.read_csv(r"C:\Users\sahab\Desktop\slovarik\сочная_фтмишница_это_все_для_тебя\сочная_фтмишница_это_все_для_тебя_4.csv")

In [4]:
data_for_forecasting_0_prepared["class"] = 0
data_for_forecasting_1_prepared["class"] = 1
data_for_forecasting_2_prepared["class"] = 2
data_for_forecasting_3_prepared["class"] = 3
data_for_forecasting_4_prepared["class"] = 4

data_for_forecasting_0_prepared["time_index"] = np.arange(368)
data_for_forecasting_1_prepared["time_index"] = np.arange(368)
data_for_forecasting_2_prepared["time_index"] = np.arange(368)
data_for_forecasting_3_prepared["time_index"] = np.arange(368)
data_for_forecasting_4_prepared["time_index"] = np.arange(368)

In [5]:
tft_df = pd.concat(
    [
        data_for_forecasting_0_prepared,
        data_for_forecasting_1_prepared,
        data_for_forecasting_2_prepared,
        data_for_forecasting_3_prepared,
        data_for_forecasting_4_prepared
    ], ignore_index=True
)

In [6]:
tft, answer = zdorovaya_dura(
    data=tft_df,
    batch_size=14,
    target="price",
    index_col="time_index",
    time_varying_unknown_reals=["price"],
    group_by=["class"],
    epochs=50,
    max_prediction_length=90
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 48    
4  | static_variable_selection          | VariableSelectionNetwork        | 528   
5  | encoder_variable_selection         | VariableSelectionNetwork        | 1.2 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 528   
7  | static_context_variable_selection  | GatedResid

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [12]:
time_index = pd.date_range(start='27/8/2023', periods=90)

answer_0_df = pd.Series(answer[0], index=time_index)
answer_1_df = pd.Series(answer[1], index=time_index)
answer_2_df = pd.Series(answer[2], index=time_index)
answer_3_df = pd.Series(answer[3], index=time_index)
answer_4_df = pd.Series(answer[4], index=time_index)